In [1]:
import numpy as np

In [2]:
import tensorflow as tf
import sklearn

In [4]:
train_x = []
train_y = []
count = 0
with open('CIKM2017_train/train.txt') as fhdl:
    for line in fhdl:
        count += 1
        if count % 50 == 0:
            print (float(count) / 10000,end=' '),
        linenum,label,datas = line.strip().split(',')
        label = float(label)
        datas = np.asarray(datas.split(' '),dtype=np.int8)
        train_x.append(datas)
        train_y.append(label)

0.005 0.01 0.015 0.02 0.025 0.03 0.035 0.04 0.045 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 0.09 0.095 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 0.145 0.15 0.155 0.16 0.165 0.17 0.175 0.18 0.185 0.19 0.195 0.2 0.205 0.21 0.215 0.22 0.225 0.23 0.235 0.24 0.245 0.25 0.255 0.26 0.265 0.27 0.275 0.28 0.285 0.29 0.295 0.3 0.305 0.31 0.315 0.32 0.325 0.33 0.335 0.34 0.345 0.35 0.355 0.36 0.365 0.37 0.375 0.38 0.385 0.39 0.395 0.4 0.405 0.41 0.415 0.42 0.425 0.43 0.435 0.44 0.445 0.45 0.455 0.46 0.465 0.47 0.475 0.48 0.485 0.49 0.495 0.5 0.505 0.51 0.515 0.52 0.525 0.53 0.535 0.54 0.545 0.55 0.555 0.56 0.565 0.57 0.575 0.58 0.585 0.59 0.595 0.6 0.605 0.61 0.615 0.62 0.625 0.63 0.635 0.64 0.645 0.65 0.655 0.66 0.665 0.67 0.675 0.68 0.685 0.69 0.695 0.7 0.705 0.71 0.715 0.72 0.725 0.73 0.735 0.74 0.745 0.75 0.755 0.76 0.765 0.77 0.775 0.78 0.785 0.79 0.795 0.8 0.805 0.81 0.815 0.82 0.825 0.83 0.835 0.84 0.845 0.85 0.855 0.86 0.865 0.87 0.875 0.88 0.885 0.89 0.895 0.9 0.905 0.91 0.915 0.

In [3]:
import pickle
train_x,train_y = pickle.load(open('data.pkl','rb'))

In [4]:
len(train_x),len(train_y)

(10000, 10000)

In [5]:
from sklearn.cross_validation import train_test_split
train_x,test_x,train_y,test_y = train_test_split(train_x,train_y,test_size=0.2,random_state=1123)

In [6]:
# do not use this now, will cause future shuffle slow
#train_x = np.asarray(train_x,dtype=np.int8)
test_x = np.asarray(test_x,dtype=np.int8)

In [9]:
len(train_x),len(train_y),test_x.shape,len(test_y)

(8000, 8000, (2000, 612060), 2000)

In [8]:
from sklearn.utils import shuffle
train_x,train_y = shuffle(train_x,train_y)

In [ ]:
import pickle
pickle.dump((train_x,train_y),open('data.pkl','wb'))

In [11]:
import sys
sys.getsizeof(train_x[0]) * 10000

960000

In [29]:
# TODO random shuffle after each epoch
# modify negative values to zero
feed_size = 100
n_input = 101 * 101 * 60
X = tf.placeholder(tf.int8,[None,n_input])
W = tf.Variable(tf.zeros([n_input,1]),tf.float32)
b = tf.Variable([0.0],tf.float32)
y = tf.add(tf.matmul(tf.to_float(X),W),b)
y_ = tf.placeholder(tf.float32)
resm = tf.reduce_sum(tf.square(tf.transpose(y) - y_))

val_train_size = 1000

train_resm = tf.sqrt(tf.div(tf.reduce_sum(tf.square(tf.transpose(y) - y_)), val_train_size))
real_resm = tf.sqrt(tf.div(tf.reduce_sum(tf.square(tf.transpose(y) - y_)), len(test_y)))

train_step = tf.train.AdamOptimizer(0.0000003).minimize(resm)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

for i in range(4000):
    indexbegin = feed_size * i % len(train_y)
    batch_xs,batch_ys = train_x[indexbegin:indexbegin + feed_size,:],train_y[indexbegin:indexbegin + feed_size]
    if i % 40 == 0:
        print('itertion %s' % (i),end=' ')
        curr_loss = sess.run(train_resm,feed_dict={X:train_x[:val_train_size],y_:train_y[:val_train_size]})
        print ("train loss: %s" % (curr_loss),end=' ')
        curr_loss = sess.run(real_resm,feed_dict={X:test_x,y_:test_y})
        print ("test loss: %s" % (curr_loss))
    sess.run(train_step,feed_dict={X:batch_xs,y_:batch_ys})

Instructions for updating:
Use `tf.global_variables_initializer` instead.
itertion 0 train loss: 21.5276 test loss: 22.2155
itertion 40 train loss: 16.4931 test loss: 17.1664
itertion 80 train loss: 15.9607 test loss: 16.5217
itertion 120 train loss: 15.0604 test loss: 16.0855
itertion 160 train loss: 15.2414 test loss: 16.0495
itertion 200 train loss: 14.3251 test loss: 15.6556
itertion 240 train loss: 14.6419 test loss: 15.7629
itertion 280 train loss: 13.7675 test loss: 15.3996
itertion 320 train loss: 14.1497 test loss: 15.5732
itertion 360 train loss: 13.311 test loss: 15.2301
itertion 400 train loss: 13.7222 test loss: 15.4347
itertion 440 train loss: 12.919 test loss: 15.1122
itertion 480 train loss: 13.3409 test loss: 15.3304
itertion 520 train loss: 12.5704 test loss: 15.0273
itertion 560 train loss: 12.9942 test loss: 15.2501
itertion 600 train loss: 12.2531 test loss: 14.9646
itertion 640 train loss: 12.6738 test loss: 15.1869
itertion 680 train loss: 11.9593 test loss: 14.9

KeyboardInterrupt: 

In [12]:
sample = np.asarray([-2,2],dtype=np.int8)

In [13]:
sample[sample < 0] = 0
sample

array([0, 2], dtype=int8)

In [14]:
# process negative value
for each_ele in train_x:
    each_ele[each_ele < 0] = 0
test_x[test_x < 0] = 0

In [15]:
len(train_x),test_x.shape

(8000, (2000, 612060))

In [23]:
# TODO use 100% data ,not 80% now

weights_dic = {}

from sklearn.utils import shuffle
feed_size = 100
n_input = 101 * 101 * 60
X = tf.placeholder(tf.int8,[None,n_input])
W = tf.Variable(tf.zeros([n_input,1]),tf.float32)
b = tf.Variable([0.0],tf.float32)
y = tf.add(tf.matmul(tf.to_float(X),W),b)
y_ = tf.placeholder(tf.float32)
resm = tf.reduce_sum(tf.square(tf.transpose(y) - y_))

val_train_size = 1000

train_resm = tf.sqrt(tf.div(tf.reduce_sum(tf.square(tf.transpose(y) - y_)), val_train_size))
real_resm = tf.sqrt(tf.div(tf.reduce_sum(tf.square(tf.transpose(y) - y_)), len(test_y)))

train_step = tf.train.AdamOptimizer(0.0000003).minimize(resm)
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

for i in range(4000):
    indexbegin = feed_size * i % len(train_y)
    if i % (len(train_y) / (feed_size)) == 0:
        print("shuffleing ...")
        train_x,train_y = shuffle(train_x,train_y)
    batch_xs,batch_ys = np.asarray(train_x[indexbegin:indexbegin + feed_size],dtype=np.int8),train_y[indexbegin:indexbegin + feed_size]
    if i % (len(train_y) / (feed_size * 2)) == 0:
        print('itertion %s' % (i),end=' ')
        curr_loss = sess.run(train_resm,feed_dict={X:train_x[:val_train_size],y_:train_y[:val_train_size]})
        print ("train loss: %s" % (curr_loss),end=' ')
        curr_loss = sess.run(real_resm,feed_dict={X:test_x,y_:test_y})
        print ("test loss: %s" % (curr_loss),end=" ")
        print(" check weights_dic[%s]" % (i))
        weights_dic[i] = (sess.run(W),sess.run(b))
    sess.run(train_step,feed_dict={X:batch_xs,y_:batch_ys})

Instructions for updating:
Use `tf.global_variables_initializer` instead.
shuffleing ...
itertion 0 train loss: 22.0375 test loss: 22.2155  check weights_dic[0]
itertion 40 train loss: 16.8781 test loss: 17.1524  check weights_dic[40]
shuffleing ...
itertion 80 train loss: 16.1381 test loss: 16.6459  check weights_dic[80]
itertion 120 train loss: 15.7058 test loss: 16.5028  check weights_dic[120]
shuffleing ...
itertion 160 train loss: 16.2856 test loss: 16.1613  check weights_dic[160]
itertion 200 train loss: 15.7893 test loss: 16.0358  check weights_dic[200]
shuffleing ...
itertion 240 train loss: 15.6402 test loss: 15.806  check weights_dic[240]
itertion 280 train loss: 15.3826 test loss: 15.7308  check weights_dic[280]
shuffleing ...
itertion 320 train loss: 15.4396 test loss: 15.8067  check weights_dic[320]
itertion 360 train loss: 14.7605 test loss: 15.5207  check weights_dic[360]
shuffleing ...
itertion 400 train loss: 14.5223 test loss: 15.5137  check weights_dic[400]
itertion 

KeyboardInterrupt: 

In [25]:
import pickle
with open('data/4_11_tf_linear_adam_80_precent_weight.plk','wb') as whdl:
    pickle.dump(weights_dic,whdl)

In [26]:
weights_dic.keys()

dict_keys([0, 1280, 1560, 2600, 520, 600, 1120, 1040, 1880, 280, 1320, 2080, 560, 40, 240, 2320, 3360, 3120, 3080, 1080, 1600, 480, 1920, 880, 1400, 840, 2440, 80, 3000, 2560, 3160, 1440, 1840, 2400, 1680, 2880, 3400, 2920, 1640, 3200, 720, 1240, 2160, 360, 1760, 3320, 120, 200, 2280, 640, 320, 1160, 1800, 2960, 920, 2680, 2200, 2640, 2720, 2800, 3240, 1200, 2120, 440, 960, 800, 1960, 1480, 160, 2240, 2000, 760, 1360, 2520, 2360, 3040, 3280, 1000, 2840, 1520, 680, 400, 2480, 2040, 1720, 2760])

In [31]:
weights = weights_dic[1280]

(array([[  2.01886705e-05],
        [  2.55818595e-05],
        [  1.79752569e-05],
        ..., 
        [  5.49545511e-08],
        [ -9.89769433e-06],
        [ -8.75896967e-06]], dtype=float32),
 array([ 0.00021187], dtype=float32))

In [32]:
del train_x
del train_y
del test_x
del test_y

In [33]:
predict_x = []
count = 0
with open('CIKM2017_testA/testA.txt') as fhdl:
    for line in fhdl:
        count += 1
        if count % 50 == 0:
            print (float(count) / 10000,end=' '),
        linenum,label,datas = line.strip().split(',')
        label = float(label)
        datas = np.asarray(datas.split(' '),dtype=np.int8)
        predict_x.append(datas)

0.005 0.01 0.015 0.02 0.025 0.03 0.035 0.04 0.045 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 0.09 0.095 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 0.145 0.15 0.155 0.16 0.165 0.17 0.175 0.18 0.185 0.19 0.195 0.2 

In [34]:
with open('onlinedata.plk','wb') as whdl:
    pickle.dump(predict_x,whdl)

In [70]:
predict_y = sess.run(y,feed_dict={X:np.asarray(predict_x,dtype=np.int8)})

In [71]:
predict_y

array([[  7.89219189],
       [ 10.65508747],
       [ 12.67513371],
       ..., 
       [  5.09071112],
       [  2.88096619],
       [  3.77302599]], dtype=float32)

In [35]:
model_w,model_b = weights_dic[1280]

In [42]:
mm = np.dot(np.asarray(predict_x),model_w) + model_b

In [52]:
mm[:,0]

array([ 14.0964489 ,  13.99987507,  14.10657501, ...,  18.26054955,
         5.55170965,  13.96566772], dtype=float32)

In [60]:
for itern in range(0,1000,160):
    print(itern)
    with open('answers/4_11_tf_linear_adam_80_precent_weight_iter_%s.csv' % itern,'w') as whdl:
        model_w,model_b = weights_dic[itern]
        predict_y = np.dot(np.asarray(predict_x),model_w) + model_b
        for value in predict_y[:,0]:
            whdl.write("%s\n" % (max(value,0)))
for itern in range(1000,2760,40):
    print (itern)
    with open('answers/4_11_tf_linear_adam_80_precent_weight_iter_%s.csv' % itern,'w') as whdl:
        model_w,model_b = weights_dic[itern]
        predict_y = np.dot(np.asarray(predict_x),model_w) + model_b
        for value in predict_y[:,0]:
            whdl.write("%s\n" % (max(value,0)))
        

0
160
320
480
640
800
960
1000
1040
1080
1120
1160
1200
1240
1280
1320
1360
1400
1440
1480
1520
1560
1600
1640
1680
1720
1760
1800
1840
1880
1920
1960
2000
2040
2080
2120
2160
2200
2240
2280
2320
2360
2400
2440
2480
2520
2560
2600
2640
2680
2720


In [74]:
predict_y[0][0]

7.8921919

In [75]:
with open('answer_4_11.csv','w') as whdl:
    for value in predict_y:
        whdl.write("%s\n" % (value[0]))